# Pattern recognition CW2

In [1]:
from scipy.io import loadmat
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import time

### Import file with 6 main components:

In [2]:
""" 
camId : which camera was used to get the shot (1 or 2)
filelist: names of the images (with format x_label_camId_index.png)
labels: class of the image (which person's image is it?)
query_idx: indexes of test set
gallery_idx: indexes of test set used for kNN 
train_idx: indexes of training and validation set
"""   
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')

### Import file with the feature vectors of all images:

In [3]:
import json
with open('PR_data/feature_data.json', 'r') as f:
    features = json.load(f)
features_np = np.array(features) #list of features converted to an array 
features_np.shape

(14096, 2048)

In [4]:
np.unique(train_idxs['labels'])[-1]

1467

In [5]:
train_idxs['gallery_idx'].shape

(5328, 1)

In [6]:
train_idxs['train_idx']

array([[    1],
       [    2],
       [    3],
       ...,
       [14094],
       [14095],
       [14096]], dtype=uint16)

In [7]:
7368+5328

12696

In [8]:
train_idxs['query_idx'].shape

(1400, 1)

In [9]:
12696+1400

14096

## kNN Classification for query set on gallery set

In [11]:
def extract_info_from_filename(image_index): 
    name = str(train_idxs['filelist'][image_index][0][0])
    [dc, label, camId, index] = name.split('_')
    return label, camId

def delete_images(query_image, query_index, gallery_images, gallery_labels):
    gallery_names = train_idxs['filelist'][train_idxs['gallery_idx'].flatten()]
    query_label, query_camId = extract_info_from_filename(query_index)
    new_gallery_images = []
    new_gallery_labels = []
    for img, name in zip(gallery_images, gallery_names):
        [dc, label_g, camId_g, index] = str(name[0][0]).split('_')
        if [int(label_g), int(camId_g)] != [int(query_label), int(query_camId)]:
            new_gallery_images.append(img)
            new_gallery_labels.append(int(label_g))
    return new_gallery_images, new_gallery_labels

def score_rank(query_label, rank_k, new_gallery_labels):
    #print(rank_k)
    rank_k_labels = np.array(new_gallery_labels)[rank_k] #extract the labels which are of rank k
    return (query_label[0] in rank_k_labels) #return true if query label in rank k, else false

In [12]:
train_idxs['gallery_idx'].flatten()

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [13]:
def NNClassification_deletions(query_image, query_index, gallery_images, gallery_labels, k):
    neigh = NearestNeighbors(n_neighbors=k, n_jobs=-1)
    new_gallery_images, new_gallery_labels = delete_images(query_image, query_index, gallery_images, gallery_labels)
    neigh.fit(new_gallery_images)   #new_gallery_labels)
    distances, indices = neigh.kneighbors(query_image, k)
    return distances, indices, new_gallery_labels #neigh.score(query_image, query_label), neigh.predict(query_image), 

In [14]:
gallery_labels = train_idxs['labels'][train_idxs['gallery_idx'].flatten()]
query_labels = train_idxs['labels'][train_idxs['query_idx'].flatten()]

query_images = features_np[train_idxs['query_idx'].flatten()]
gallery_images = features_np[train_idxs['gallery_idx'].flatten()]

def get_accuracy_for_k_ranks(k):
    list_of_truths = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()):
        if(index < 101):
            if (index % 10 == 0):
                print("Index: ", index, " & time taken: ", time.time() - start)

            #perform NN Classification and extract top k ranks 
            dist, rank_k_indices, new_gallery_labels = NNClassification_deletions(query_images[index].reshape(1, 2048), query_index, gallery_images, gallery_labels, k)
            #calculate the score (true/false) for top k images and query image recognition
            score = score_rank(query_labels[index], rank_k_indices, new_gallery_labels)
            #create a list of scores to get overall accuracy later on
            list_of_truths.append(score)
    
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)

In [15]:
train_idxs['gallery_idx'].flatten()

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [16]:
train_idxs['filelist'][204]

array([array(['1_022_2_06.png'], dtype='<U14')], dtype=object)

In [17]:
query_images

array([[0.98361254, 0.33195475, 0.26549727, ..., 0.02350602, 0.08491972,
        0.00514889],
       [1.30696166, 0.22219124, 0.62375975, ..., 0.02954952, 0.37348923,
        0.05303315],
       [0.29322624, 0.55588913, 0.06961903, ..., 0.15892459, 0.21527511,
        0.09551907],
       ...,
       [0.7593497 , 0.24537075, 2.19513273, ..., 0.47061139, 1.35971546,
        0.46198806],
       [0.48667279, 0.49618778, 0.30824399, ..., 0.04450084, 1.28799629,
        0.37683338],
       [0.05719076, 0.60635394, 0.58300596, ..., 0.40994745, 1.37061644,
        0.55975795]])

In [21]:
gallery_labels = train_idxs['labels'][train_idxs['gallery_idx'].flatten()]
query_labels = train_idxs['labels'][train_idxs['query_idx'].flatten()]

query_images = features_np[train_idxs['query_idx'].flatten()]
gallery_images = features_np[train_idxs['gallery_idx'].flatten()]

N=100


def remove_indices(n_indices, query_index):
    query_label, query_camId = extract_info_from_filename(query_index)
    n_names = train_idxs['filelist'][n_indices]
    #k_images = features_np[k_indices]
    #print(k_names.shape, k_names[0], k_images[0])
    final_n_indices = []
    final_n_labels = []
    for index, name in zip(n_indices, n_names):
        [dc, label_g, camId_g, index_g] = str(name[0][0]).split('_')
        #print(str(name[0][0]).split('_'))
        #print(index)
        if [int(label_g), int(camId_g)] != [int(query_label), int(query_camId)]:
            final_n_indices.append(index)
            final_n_labels.append(int(label_g))
    return final_n_indices, final_n_labels

def get_accuracy(k, N):
    list_of_truths = []
    start = time.time() #time tracking - start time of process
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1, metric='euclidean')
    neigh.fit(gallery_images)   #new_gallery_labels)
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()):
        if (index % 10 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        N_distances, N_indices = neigh.kneighbors(query_images[index].reshape(1, 2048), N)
        #print(query_labels[index], query_index, N_indices)
        topN_gallery_images = gallery_images[N_indices[0]]
        forLoopStart = time.time()
        topN_gallery_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
        #topN_gallery_indices = [np.where(np.all(features_np == image, axis=1))[0][0] for image in topN_gallery_images]
        #print(time.time()-forLoopStart)
        #print('topN indices', topN_gallery_indices.flatten().tolist())
        #print('n_indices', N_indices[0])
        #print(train_idxs['gallery_idx'][N_indices[0]])
        reduced_topN_indices, reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
        if query_labels[index][0] in reduced_topN_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
        if index == 300:
            break
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)
    print(acc)

In [22]:
features_np[[2, 3]].shape

(2, 2048)

In [23]:
%prun get_accuracy(1, 100)

Index:  0  & time taken:  0.6633329391479492
Index:  10  & time taken:  1.7301599979400635
Index:  20  & time taken:  2.7969608306884766
Index:  30  & time taken:  3.8662068843841553
Index:  40  & time taken:  4.922598123550415
Index:  50  & time taken:  5.985350131988525
Index:  60  & time taken:  7.051974058151245
Index:  70  & time taken:  8.112435102462769
Index:  80  & time taken:  9.16952109336853
Index:  90  & time taken:  10.236074924468994
Index:  100  & time taken:  11.31096601486206
Index:  110  & time taken:  12.360049962997437
Index:  120  & time taken:  13.427722930908203
Index:  130  & time taken:  14.492151021957397
Index:  140  & time taken:  15.546682119369507
Index:  150  & time taken:  16.605628967285156
Index:  160  & time taken:  17.666258096694946
Index:  170  & time taken:  18.732553958892822
Index:  180  & time taken:  19.792661905288696
Index:  190  & time taken:  20.844604969024658
Index:  200  & time taken:  21.90824007987976
Index:  210  & time taken:  22.9

## K-means

In [28]:
from sklearn.cluster import KMeans

total_number_of_labels = np.unique(train_idxs['labels'])[-1]
n_clusters = 3 #total_number_of_labels

kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(gallery_images)
#print(kmeans.labels_)
index_and_cluster = [(index, label) for index, label in zip(train_idxs['gallery_idx'].flatten(), kmeans.labels_)]
#np.unique(kmeans.labels_)

In [29]:
def img_indices_in_cluster(c): 
    cluster_labels = [pair[0] for pair in index_and_cluster if pair[1] == c]
    return cluster_labels

In [30]:
def img_cluster(k, N, c_g_indices, query_index, index):
    list_of_truths = []
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1, metric='euclidean')
    neigh.fit(features_np[c_g_indices])   #new_gallery_labels)
    
    
    N_distances, N_indices = neigh.kneighbors(features_np[query_index].reshape(1, -1), N)
    #print(query_labels[index], query_index, N_indices)
    #topN_c_g_images = features_np[c_g_indices][N_indices[0]]
    topN_c_g_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
    
    reduced_topN_indices, reduced_topN_labels = remove_indices(topN_c_g_indices, query_index)
    
    return (query_labels[index][0] in reduced_topN_labels[:k])

def knn_clustering(k, N):
    class_success = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()):
        if (index % 10 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        cluster = kmeans.predict(query_images[index].reshape(1,-1))
        cluster_gallery_indices = img_indices_in_cluster(cluster)
        class_success.append(img_cluster(k, N, cluster_gallery_indices, query_index, index))
        
        
        if index == 100:
            break
     #over-all accuracy
    acc = sum(class_success)/len(class_success)
    print(acc)

In [31]:
k = 10
N = 100
knn_clustering(k, N)

Index:  0  & time taken:  0.00043892860412597656
Index:  10  & time taken:  2.5827829837799072
Index:  20  & time taken:  5.329355955123901
Index:  30  & time taken:  7.973233938217163
Index:  40  & time taken:  10.791098833084106
Index:  50  & time taken:  13.536258935928345
Index:  60  & time taken:  16.281173944473267
Index:  70  & time taken:  18.76628875732422
Index:  80  & time taken:  21.52734112739563
Index:  90  & time taken:  24.169163942337036
Index:  100  & time taken:  26.962733030319214
0.039603960396039604


### Mahalanobi Distance

Getting distances from the average gallery vector:

In [39]:
#number of unique labels in gallery 
len(np.unique(gallery_labels))

977

In [63]:
S = np.cov(np.transpose(gallery_images))
A = np.linalg.inv(S)

In [64]:
A.shape

(2048, 2048)

In [113]:
gallery_images.shape

(5328, 2048)

In [69]:
mean_gallery_images = gallery_images.mean(axis=0)     # to take the mean of each column (colummn -> feature)
mean_gallery_images.shape

(2048,)

In [79]:
query_set_distances = [np.sqrt(np.dot(np.dot(np.transpose(query_image-mean_gallery_images),A), query_image-mean_gallery_images))
                      for query_image in query_images]
np.array(query_set_distances).shape

(1400,)

Getting distances using kNN and Mahalanobi Distance

In [92]:
neigh_md = NearestNeighbors(n_neighbors=N,metric='mahalanobis', 
            metric_params= {'V': np.cov(np.transpose(gallery_images))})
neigh_md.fit(gallery_images) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='mahalanobis',
         metric_params={'V': array([[ 0.09833, -0.00735, ..., -0.00826, -0.00833],
       [-0.00735,  0.20639, ..., -0.00024, -0.00667],
       ...,
       [-0.00826, -0.00024, ...,  0.22353,  0.06679],
       [-0.00833, -0.00667, ...,  0.06679,  0.10489]])},
         n_jobs=1, n_neighbors=100, p=2, radius=1.0)

In [117]:
def get_accuracy_mahalanobi(k, N):
    list_of_truths = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()[:300]):
        if (index % 50 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        N_distances, N_indices = neigh_md.kneighbors(query_images[index].reshape(1, 2048), N)
        topN_gallery_images = gallery_images[N_indices[0]]
        forLoopStart = time.time()
        topN_gallery_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
        reduced_topN_indices, reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
        if query_labels[index][0] in reduced_topN_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)
    print("Avg Accuracy is: ", acc)
    return acc

In [118]:
N = 100
k = [1, 10, 100]
acc_k2 = []
for k_val in k: 
    acc = get_accuracy_mahalanobi(k_val, N)
    acc_k2.append(acc)

Index:  0  & time taken:  8.082389831542969e-05
Index:  50  & time taken:  1194.6821658611298
Index:  100  & time taken:  2801.5326719284058
Index:  150  & time taken:  4013.9011237621307
Index:  200  & time taken:  5203.121114730835
Index:  250  & time taken:  6391.557485818863
Avg Accuracy is:  0.13666666666666666
Index:  0  & time taken:  7.224082946777344e-05
Index:  50  & time taken:  1187.0171492099762
Index:  100  & time taken:  2373.66286110878
Index:  150  & time taken:  3560.3639900684357
Index:  200  & time taken:  4746.637277126312
Index:  250  & time taken:  5932.8617470264435
Avg Accuracy is:  0.2833333333333333
Index:  0  & time taken:  9.083747863769531e-05
Index:  50  & time taken:  1186.3573276996613
Index:  100  & time taken:  2372.715752840042
Index:  150  & time taken:  3559.498749732971
Index:  200  & time taken:  4745.957722663879
Index:  250  & time taken:  5932.393654823303
Avg Accuracy is:  0.4533333333333333


In [112]:
acc_k

[0.09090909090909091, 0.18181818181818182, 0.36363636363636365]

In [119]:
acc_k2

[0.13666666666666666, 0.2833333333333333, 0.4533333333333333]

### Cosine Distance

In [120]:
N = 100
neigh_c = NearestNeighbors(n_neighbors=N,metric='cosine')
neigh_c.fit(gallery_images) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=100, p=2, radius=1.0)

In [126]:
def get_acc_gen(k, N, index, query_index):
    N_distances, N_indices = neigh_c.kneighbors(query_images[index].reshape(1, 2048), N)
    topN_gallery_images = gallery_images[N_indices[0]]
    forLoopStart = time.time()
    topN_gallery_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
    reduced_topN_indices, reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
    return (query_labels[index][0] in reduced_topN_labels[:k])

k = [1, 10, 100]
acc_k3 = []
for k_val in k:
    list_of_truths = [] 
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()[:300]):
        if (index % 50 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        truth_false = get_acc_gen(k_val, N, index, query_index)
        list_of_truths.append(truth_false)
    acc = sum(list_of_truths)/len(list_of_truths)
    print("Avg Accuracy is: ", acc)
    acc_k3.append(acc)

Index:  0  & time taken:  7.700920104980469e-05
Index:  50  & time taken:  4.227119207382202
Index:  100  & time taken:  8.423174142837524
Index:  150  & time taken:  12.630351066589355
Index:  200  & time taken:  16.83267116546631
Index:  250  & time taken:  21.04805612564087
Avg Accuracy is:  0.49
Index:  0  & time taken:  5.1975250244140625e-05
Index:  50  & time taken:  4.1925787925720215
Index:  100  & time taken:  8.413705825805664
Index:  150  & time taken:  12.6040678024292
Index:  200  & time taken:  16.792085886001587
Index:  250  & time taken:  20.98653793334961
Avg Accuracy is:  0.74
Index:  0  & time taken:  5.3882598876953125e-05
Index:  50  & time taken:  4.198216199874878
Index:  100  & time taken:  8.467583894729614
Index:  150  & time taken:  12.67863917350769
Index:  200  & time taken:  16.884459972381592
Index:  250  & time taken:  21.093831062316895
Avg Accuracy is:  0.91
